In [1]:
from numpy import *
from scipy.sparse import *
from sklearn.preprocessing import *

In [2]:
PlTrackSim = load_npz("target_pl_x_target_tracks_similarity.npz")

In [3]:
PlwithArtist = load_npz("targetPlaylistArtistReduced.npz")

In [4]:
tTracksWithArtist = load_npz("targetTracksArtistReduced.npz")

In [5]:
PlwithArtist = normalize(PlwithArtist, norm="max")

In [6]:
PlTrackSimByArtist = PlwithArtist * tTracksWithArtist.T

In [7]:
artistsTags = load_npz("artist_with_tag_bincount_reduced.npz")
artistsTags = normalize(artistsTags)

In [8]:
P=load_npz("playlist_reduced.npz")
P=normalize(P)

In [9]:
PlArtistSim = P * artistsTags.T

In [10]:
PlTrackSimByArtistTags = PlArtistSim * tTracksWithArtist.T

In [12]:
finalPlSimH = PlTrackSimByArtist + PlTrackSim 

In [13]:
finalPlSim = finalPlSimH + PlTrackSimByArtistTags

In [ ]:
finalPlSim = PlTrackSim.multiply(PlTrackSimByArtist) 

In [14]:
finalPlSim

<10000x32195 sparse matrix of type '<class 'numpy.float64'>'
	with 257059946 stored elements in Compressed Sparse Row format>

In [15]:
target_playlists=genfromtxt("target_playlists.csv",skip_header=1)
playlists_with_tracks=load("playlists_with_tracks.npy")
target_tracks_ordered = load("targetTracksOrdered.npy")
def getsimil(pls, similrow):
    maxi = flip(argsort(similrow), axis=0)
    r = []
    for m in maxi:
        if(not isin(target_tracks_ordered[m], pls[1:])):
           r.append(target_tracks_ordered[m])
           if(len(r)==5):
               return r

In [16]:
fname = "tag_artist2_average_similarity.csv"
open(fname,"w").close()
print("playlist_id,track_ids",file=open(fname,"a"))
for pl, simil in zip(playlists_with_tracks, finalPlSim):
    s = str(int(pl[0]))
    s += ","
    r = getsimil(pl, ravel(simil.todense()))
    for el in r:
        s+=str(el)
        s+=" "
    print(s,file=open(fname,"a"))

In [17]:
k = asarray([[3, 0, 1, 0], [1, 1, 2, 0], [0, 0, 1, 3]])

In [22]:
k =normalize(k)

In [23]:
z = asarray([[1, 0, 1, 0], [0, 1, 1, 0], [0, 1, 0, 1]])

In [24]:
dot(k, z.T)

array([[ 1.26491106,  0.31622777,  0.        ],
       [ 1.22474487,  1.22474487,  0.40824829],
       [ 0.31622777,  0.31622777,  0.9486833 ]])